In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from IPython.display import HTML

### NASA Mars News

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
sleep(5)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\foong\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache


In [3]:
# Create a Beautiful Soup object; parse with 'html.parser'
soup = bs(browser.html, "html.parser")

In [4]:
# Results are returned as an iterable list
results = soup.find_all("li",class_="slide")

In [5]:
# Extract title from the first result in list for the latest article
news_title = results[0].find("div",class_="content_title").text
print(news_title)

NASA Establishes Board to Initially Review Mars Sample Return Plans


In [6]:
# Extract paragraph from the first result in list for the latest article
news_p = results[0].find("div",class_="article_teaser_body").text
print(news_p)

The board will assist with analysis of current plans and goals for one of the most difficult missions humanity has ever undertaken.


### JPL Mars Space Images - Featured Image

In [7]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
sleep(1)

In [8]:
# Create a Beautiful Soup object; parse with 'html.parser'
soup = bs(browser.html, "html.parser")

In [9]:
# Discover all attributes contained in tag <article> to find attribute containing image url
soup.article.attrs

{'alt': "Crystal-Laden Martian Rock Examined by Curiosity's Laser Instrument",
 'class': ['carousel_item'],
 'style': "background-image: url('/spaceimages/images/wallpaper/PIA17767-1920x1200.jpg');"}

In [10]:
# Assign url for full size .jpeg image to a variable
transform = soup.article["style"]

# Find start and end position of url in string
start = transform.rfind("url('") + len("url('") 
end = transform.rfind("')")

# Save full image url in variable
main_url = "https://www.jpl.nasa.gov"
featured_image_url = main_url + transform[start:end]
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17767-1920x1200.jpg


### Mars Facts

In [11]:
url = "https://space-facts.com/mars/"

In [12]:
# Use pandas 'read_html' function to scrape all tabular data from a page
tables = pd.read_html(url)
print("Classification of object returned:",type(tables))
print("Number of dataframes in list:",len(tables))

Classification of object returned: <class 'list'>
Number of dataframes in list: 3


In [13]:
# Choose the first dataframe which contains the Mars Planet Profile table using normal indexing
mars_planet_profile_df = tables[0]
mars_planet_profile_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [14]:
# Clean up table
mars_planet_profile_df.rename(columns={0: "", 1: "Mars"}, inplace=True) # Rename column header
mars_planet_profile_df.set_index([""], inplace=True) # Set column 0 as index 
mars_planet_profile_df

,Mars
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
# Use pandas 'to_html' function to convert dataframe to HTML table string
mars_planet_profile_html = mars_planet_profile_df.to_html()
print("Classification of object returned:",type(mars_planet_profile_html))

Classification of object returned: <class 'str'>


In [16]:
# Amend string to remove index header row and use HTML to render table in notebook
mars_planet_profile_html = mars_planet_profile_html.replace("    <tr>\n      <th></th>\n      <th></th>\n    </tr>\n","")
HTML(mars_planet_profile_html)

,Mars
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [17]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
sleep(1)

In [18]:
# Create a Beautiful Soup object; parse with 'html.parser'
soup = bs(browser.html, "html.parser")

In [19]:
# Save the results for each hemisphere in a list called "items"
items = soup.find_all("div",class_="item")
len(items)

4

In [20]:
# Loop: Get the links to the hemispheres pages and save it into a list
image_destinations = list()
main_url = "https://astrogeology.usgs.gov"

for item in items:
    link = list(item.children)[0]["href"]
    image_destinations.append(main_url + link)

len(image_destinations)

4

In [21]:
# Loop: Use splinter to navigate to each hemisphere page and create dictionary with image title and url (full res)
hemisphere_image_urls = []

for image_destination in image_destinations:
    browser.visit(image_destination)
    sleep(1)
    soup = bs(browser.html, "html.parser")
    title = soup.h2.text.replace(" Enhanced","")
    img_url = soup.li.a["href"]
    hemisphere_image_urls.append({"title": title, "img_url": img_url})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]